In [ ]:
#Code that results average throughput per time, for each ‘Co Server’ and ASN

In [157]:
'''
Saved the given data into a data file and extracted into a rdd.
From the rdd created 3 more RDDs in key,value pair as below
- co-server, time  
- asn,time 
- (co-server,asn),time
Apllied combineByKey on both rdd and saved it into a list.
Wrote the lists to 3 separate files
- ASN.data
- Co-server.data
- Combined.txt
'''

'\nSaved the given data into a data file and extracted into a rdd.\nFrom the rdd created 3 more RDDs in key,value pair as below\n- co-server, time  \n- asn,time \n- (co-server,asn),time\nApllied combineByKey on both rdd and saved it into a list.\nWrote the lists to 3 separate files\n- ASN.data\n- Co-server.data\n- Combined.txt\n'

In [158]:
sc = SparkContext.getOrCreate()
mydata = sc.textFile("BigData.data")

In [159]:
mydata.take(5)

[u'de 31334 [07/Aug/2015:20:30:01 +0000] 76410 302 435 0.326',
 u'pl 50231 [07/Aug/2015:20:30:01 +0000] 126746 200 7400 0.000',
 u'gr 3329 [07/Aug/2015:20:30:01 +0000] 126474 206 17711 0.000',
 u'tr 9121 [07/Aug/2015:20:30:02 +0000] 76406 200 19589 0.000',
 u'se 3301 [07/Aug/2015:20:30:02 +0000] 76406 200 17960 0.000']

In [160]:
rdd1 = mydata.map(lambda x:x.split())

In [161]:
print rdd1.take(5)

[[u'de', u'31334', u'[07/Aug/2015:20:30:01', u'+0000]', u'76410', u'302', u'435', u'0.326'], [u'pl', u'50231', u'[07/Aug/2015:20:30:01', u'+0000]', u'126746', u'200', u'7400', u'0.000'], [u'gr', u'3329', u'[07/Aug/2015:20:30:01', u'+0000]', u'126474', u'206', u'17711', u'0.000'], [u'tr', u'9121', u'[07/Aug/2015:20:30:02', u'+0000]', u'76406', u'200', u'19589', u'0.000'], [u'se', u'3301', u'[07/Aug/2015:20:30:02', u'+0000]', u'76406', u'200', u'17960', u'0.000']]


In [162]:
rdd2 = rdd1.map(lambda x : [x[1],x[5],float(x[7])])

In [163]:
rdd2.take(20)

[[u'31334', u'302', 0.326],
 [u'50231', u'200', 0.0],
 [u'3329', u'206', 0.0],
 [u'9121', u'200', 0.0],
 [u'3301', u'200', 0.0],
 [u'9198', u'200', 0.0],
 [u'6830', u'200', 0.0],
 [u'21246', u'200', 0.0],
 [u'21246', u'200', 0.0],
 [u'9050', u'206', 0.504],
 [u'8447', u'200', 0.006],
 [u'3320', u'200', 2.892],
 [u'3320', u'200', 0.007],
 [u'3320', u'200', 2.17],
 [u'8447', u'200', 0.029],
 [u'8447', u'200', 0.0],
 [u'1901', u'206', 0.47],
 [u'3320', u'200', 0.633],
 [u'12635', u'206', 0.0],
 [u'15685', u'206', 0.756]]

In [164]:
l = rdd2.take(20)
print l

[[u'31334', u'302', 0.326], [u'50231', u'200', 0.0], [u'3329', u'206', 0.0], [u'9121', u'200', 0.0], [u'3301', u'200', 0.0], [u'9198', u'200', 0.0], [u'6830', u'200', 0.0], [u'21246', u'200', 0.0], [u'21246', u'200', 0.0], [u'9050', u'206', 0.504], [u'8447', u'200', 0.006], [u'3320', u'200', 2.892], [u'3320', u'200', 0.007], [u'3320', u'200', 2.17], [u'8447', u'200', 0.029], [u'8447', u'200', 0.0], [u'1901', u'206', 0.47], [u'3320', u'200', 0.633], [u'12635', u'206', 0.0], [u'15685', u'206', 0.756]]


In [165]:
'''dict_asn = []
for el in l:
    b = (el[0], el[2])
    dict_asn.append(b)
#print dict_asn

rdd3 = sc.parallelize(dict_asn)
rdd3.aggregateByKey(0,lambda acc,val: acc+val, lambda acc1, acc2: acc1+acc2).collect()'''
rdd3 = rdd2.map(lambda x : (x[0],x[2]))
#print rdd3.collect()
#rdd3.aggregateByKey(0,lambda acc,val: acc+val, lambda acc1, acc2: acc1+acc2).collect()
sumCount = rdd3.combineByKey(lambda value: (value, 1),
                             lambda x, value: (x[0] + value, x[1] + 1),
                             lambda x, y: (x[0] + y[0], x[1] + y[1]))

averageByKey = sumCount.map(lambda (label, (value_sum, count)): (label, value_sum / count))

list_rdd_asn = averageByKey.collect()
print list_rdd_asn

[(u'3320', 1.4255), (u'15685', 0.756), (u'21246', 0.0), (u'12635', 0.0), (u'9050', 0.504), (u'50231', 0.0), (u'9121', 0.0), (u'6830', 0.0), (u'31334', 0.326), (u'8447', 0.011666666666666667), (u'3329', 0.0), (u'9198', 0.0), (u'1901', 0.47), (u'3301', 0.0)]


In [166]:
with open('ASN.data', 'w') as fp:
    for x in list_rdd_asn:
        #print x
        fp.write(','.join(str(z) for z in x))
        fp.write('\n')
fp.close()

In [167]:
'''dict_co_server = []
count_co_server={}
for el in l:
    b = (el[1], el[2])
    count_co_server[el[1]]=sum(x.count(el[0]) for x in l)
    dict_co_server.append(b)
#print dict_co_server
#print count
rdd4 = sc.parallelize(dict_co_server)
rdd4.aggregateByKey(0,lambda acc,val: acc+val, lambda acc1, acc2: acc1+acc2).collect()'''
rdd4 = rdd2.map(lambda x : (x[1],x[2]))
rdd4.take(20)
#rdd4.aggregateByKey(0,lambda acc,val: acc+val, lambda acc1, acc2: acc1+acc2).collect()

sumCount = rdd4.combineByKey(lambda value: (value, 1),
                             lambda x, value: (x[0] + value, x[1] + 1),
                             lambda x, y: (x[0] + y[0], x[1] + y[1]))

averageByKey = sumCount.map(lambda (label, (value_sum, count)): (label, value_sum / count))

list_rdd_coserver = averageByKey.collect()
print list_rdd_coserver

[(u'302', 0.326), (u'200', 0.4097857142857143), (u'206', 0.346)]


In [168]:
with open('Co-server.data', 'w') as fp:
    for x in list_rdd_coserver:
        #print x
        fp.write(','.join(str(z) for z in x))
        fp.write('\n')
fp.close()

In [169]:
rdd_combined = rdd2.map(lambda x : ((x[0],x[1]),int(x[2])))
#print rdd_combined.take(20)
list_rdd_combined = rdd_combined.aggregateByKey(0,lambda acc,val: acc+val, lambda acc1, acc2: acc1+acc2).collect()
print list_rdd_combined

[((u'1901', u'206'), 0), ((u'6830', u'200'), 0), ((u'3329', u'206'), 0), ((u'9198', u'200'), 0), ((u'8447', u'200'), 0), ((u'9121', u'200'), 0), ((u'3301', u'200'), 0), ((u'3320', u'200'), 4), ((u'50231', u'200'), 0), ((u'9050', u'206'), 0), ((u'15685', u'206'), 0), ((u'21246', u'200'), 0), ((u'12635', u'206'), 0), ((u'31334', u'302'), 0)]


In [170]:
with open('Combined.txt', 'w') as fp:
    for x in list_rdd_combined:
        #print x[-1]
        #for y in x:
        #    print y
        fp.write(','.join(str(z) for z in x[0]))
        fp.write(',')
        fp.write(''.join(str(x[-1])))
        fp.write('\n')

In [171]:
'''
I here grouped based on co-server and asn combined for multiple key grouping.
We can group based on metric and country to get more features out of the data collected.
Metric will help in getting more insight into the data we are receiving.
'''

'\nI here grouped based on co-server and asn combined for multiple key grouping.\nWe can group based on metric and country to get more features out of the data collected.\nMetric will help in getting more insight into the data we are receiving.\n'